### Simulation of a simple refrigerator

1. Import the necessary packages

In [161]:
from tespy.networks import Network
from tespy.components import Sink, Source, SimpleHeatExchanger, Compressor, CycleCloser, Valve
from tespy.connections import Connection, Bus
from CoolProp.CoolProp import PropsSI

2. Create the `Network` for the refrigerator and set the units you want to use

In [162]:
refrigerator = Network(T_unit='C', p_unit='bar', h_unit='kJ / kg', iterinfo=False)

In [163]:
wf = 'R600a'
wf_dict = {wf: 1}

3. Set the components of the system

In [164]:
comp = Compressor('compressor')
cond = SimpleHeatExchanger('condenser')
wf_source = Source('source')
wf_sink = Sink('sink')

4. Set the connections of the system and add them to the `Network`

In [165]:
c1 = Connection(wf_source, 'out1', comp, 'in1', label='1')
c2 = Connection(comp, 'out1', cond, 'in1', label='2')
c3 = Connection(cond, 'out1', wf_sink, 'in1', label='3')

refrigerator.add_conns(c1, c2, c3)

5. Define the parameters and design variables

In [166]:
# Components
comp.set_attr(eta_s=0.75)
cond.set_attr(pr=1)

# Connections
c1.set_attr(fluid=wf_dict, T=-18, p=0.725, m=0.001)
c2.set_attr(T=60)
h3_set = PropsSI("H", "Q", 0, "T", 273.15 + 40, wf) / 1e3 
c3.set_attr(h=h3_set)

If we try to set the temperature of stream 3 to 40°C, the solver cannot find a solution. To solve this problem, we specify the enthalpy instead of the temperature. In general, enthalpies and pressure should be specified first. The value of the enthalpy (the start value, the guess value) at 40°C and at boiling condtion can be calculated using CoolProp.

6. Solve the system of equations

In [167]:
refrigerator.solve(mode='design')
refrigerator.print_results()


##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+--------+
|            |        P |       pr |        dp |    eta_s |   igva |
|------------+----------+----------+-----------+----------+--------|
| compressor | 1.11e+02 | 9.18e+00 | -5.93e+00 | 7.50e-01 |    nan |
+------------+----------+----------+-----------+----------+--------+
##### RESULTS (SimpleHeatExchanger) #####
+-----------+-----------+----------+----------+----------+-----+-----+------+---------+------+--------+
|           |         Q |       pr |       dp |     zeta |   D |   L |   ks |   ks_HW |   kA |   Tamb |
|-----------+-----------+----------+----------+----------+-----+-----+------+---------+------+--------|
| condenser | -3.45e+02 | 1.00e+00 | 0.00e+00 | 0.00e+00 | nan | nan |  nan |     nan |  nan |    nan |
+-----------+-----------+----------+----------+----------+-----+-----+------+---------+------+--------+
##### RESULTS (Connection) #####
+----+-----------+--------

Now the solver has better guesses for the next run. We can remove the specification of the enthalpy value and set the temperature at the outlet of the condenser, as required. 

In [168]:
c3.set_attr(h=None, T=40)
refrigerator.solve(mode='design')
refrigerator.print_results()


##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+--------+
|            |        P |       pr |        dp |    eta_s |   igva |
|------------+----------+----------+-----------+----------+--------|
| compressor | 1.11e+02 | 9.18e+00 | -5.93e+00 | 7.50e-01 |    nan |
+------------+----------+----------+-----------+----------+--------+
##### RESULTS (SimpleHeatExchanger) #####
+-----------+-----------+----------+----------+----------+-----+-----+------+---------+------+--------+
|           |         Q |       pr |       dp |     zeta |   D |   L |   ks |   ks_HW |   kA |   Tamb |
|-----------+-----------+----------+----------+----------+-----+-----+------+---------+------+--------|
| condenser | -3.45e+02 | 1.00e+00 | 0.00e+00 | 0.00e+00 | nan | nan |  nan |     nan |  nan |    nan |
+-----------+-----------+----------+----------+----------+-----+-----+------+---------+------+--------+
##### RESULTS (Connection) #####
+----+-----------+--------

Now, we can add the other components one by one. At the end our simulation should look like this:

In [169]:
refrigerator = Network(T_unit='C', p_unit='bar', h_unit='kJ / kg', m_unit='kg / s')

In [170]:
wf = "R600a"
wf_dict = {wf: 1}

In [171]:
val = Valve('throttling-valve')
eva = SimpleHeatExchanger('evaporator')
cond = SimpleHeatExchanger('condenser')
comp = Compressor('compressor')
cc = CycleCloser('cycle-closer')

In [172]:
c01 = Connection(eva, 'out1', comp, 'in1', label='1')
c02 = Connection(comp, 'out1', cond, 'in1', label='2')
c02cc = Connection(cond, 'out1', cc, 'in1', label='2cc')
c03 = Connection(cc, 'out1', val, 'in1', label='3')
c04 = Connection(val, 'out1', eva, 'in1', label='4')

refrigerator.add_conns(c01, c02, c02cc, c03, c04)

First run with the starting values:

In [173]:
# Components
eva.set_attr(Q=150, pr=1)
comp.set_attr(eta_s=0.75)
cond.set_attr(pr=1)

# Connections
c03.set_attr(fluid=wf_dict)
p4_set = PropsSI("P", "Q", 1, "T", 273.15 - 20, wf) / 1e5  # saturation pressure at given temperature
c04.set_attr(p=p4_set)
h1_set = PropsSI("H", "P", p4_set*1e5, "T", 273.15 - 18, wf) / 1e3  # superheated state at given temperature and pressure
c01.set_attr(h=h1_set)
c02.set_attr(T=60)
h2cc_set = PropsSI("H", "Q", 0, "T", 273.15 + 40, wf) / 1e3  # 
c02cc.set_attr(h=h2cc_set)

In [174]:
refrigerator.solve(mode='design')
refrigerator.print_results()


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 5.98e+05   | 2 %        | 1.43e+00   | 3.72e+05   | 2.04e+05   | 0.00e+00   | 0.00e+00   
 2     | 3.56e+04   | 16 %       | 2.27e-02   | 4.37e+05   | 8.20e+03   | 0.00e+00   | 0.00e+00   
 3     | 7.34e+03   | 23 %       | 1.09e-18   | 1.30e+05   | 3.22e+03   | 0.00e+00   | 0.00e+00   
 4     | 3.02e+02   | 39 %       | 0.00e+00   | 5.02e+03   | 1.79e+02   | 0.00e+00   | 0.00e+00   
 5     | 3.83e-01   | 71 %       | 0.00e+00   | 5.53e+00   | 2.66e-01   | 0.00e+00   | 0.00e+00   
 6     | 4.64e-07   | 100 %      | 0.00e+00   | 6.01e-06   | 3.52e-07   | 0.00e+00   | 0.00e+00   
 7     | 1.17e-10   | 100 %      | 0.00e+00   | 7.99e-10   | 1.32e-10   | 0.00e+00   | 0.00e+00   
Total iterations: 7, Calculation time: 0.05 s, Iterations per second: 139.61

##### RESULTS (Compressor) ###

Second run with the desired specifications:

In [175]:
c01.set_attr(h=None, Td_bp=2)
c04.set_attr(p=None, T=-20)
c02cc.set_attr(h=None, T=40)

In [176]:
refrigerator.solve(mode='design')
refrigerator.print_results()


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.86e-02   | 85 %       | 1.29e-07   | 8.38e-07   | 8.17e+01   | 0.00e+00   | 0.00e+00   
 2     | 6.10e-06   | 100 %      | 2.96e-11   | 1.21e-08   | 2.32e-03   | 0.00e+00   | 0.00e+00   
 3     | 8.85e-11   | 100 %      | 2.71e-19   | 7.52e-09   | 5.34e-10   | 0.00e+00   | 0.00e+00   
 4     | 1.75e-10   | 100 %      | 3.71e-19   | 6.25e-10   | 5.54e-10   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.13 s, Iterations per second: 29.74

##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+--------+
|            |        P |       pr |        dp |    eta_s |   igva |
|------------+----------+----------+-----------+----------+--------|
| compressor | 7.12e+01 | 9.18e+00 | -5.93e+00 | 7.50e-01 |    nan |
+------------+-----

8. Add busses for heat and power flows

In [177]:
motor = Bus('motor')
motor.add_comps({'comp': comp, 'char': 0.95, 'base': 'bus'})

cooling = Bus('cooling heat')
cooling.add_comps({'comp': eva, 'base': 'bus'})

refrigerator.add_busses(motor, cooling)

9. Run again the simulation and calculate the coefficient of performance

In [178]:
refrigerator.solve(mode='design')

print('COP:', cooling.P.val / motor.P.val)


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.46e-11   | 100 %      | 2.18e-19   | 1.80e-09   | 8.83e-11   | 0.00e+00   | 0.00e+00   
 2     | 8.85e-11   | 100 %      | 4.86e-19   | 1.69e-09   | 1.81e-10   | 0.00e+00   | 0.00e+00   
 3     | 2.04e-10   | 100 %      | 9.39e-20   | 2.32e-09   | 9.38e-11   | 0.00e+00   | 0.00e+00   
 4     | 2.33e-10   | 100 %      | 3.37e-19   | 1.17e-09   | 1.61e-10   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.10 s, Iterations per second: 40.34
COP: 2.0025168800023607
